In [ ]:
import requests
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.llms.base import LLM
from langchain.prompts import PromptTemplate
from pydantic import BaseModel, Field
from typing import Optional, List

# Sample company policy documents
documents = [
    "Refund Policy: We offer a full refund within 30 days of purchase if you are not satisfied with the product.",
    "To request a refund, please contact customer support with your order details.",
    "Refunds will be processed within 7 business days.",
    "Products must be returned in original condition."
]

# Create embeddings and FAISS index
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = FAISS.from_texts(documents, embedding_model)

# Custom LLM wrapper for OpenRouter API
class OpenRouterLLM(LLM, BaseModel):
    api_key: str = Field(..., exclude=True)
    model_name: str = "gpt-4o-mini"
    temperature: float = 0.7
    max_tokens: int = 150
    top_p: float = 0.9

    @property
    def _llm_type(self) -> str:
        return "openrouter"

    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        url = "https://openrouter.ai/api/v1/chat/completions"
        headers = {
            "Authorization": f"Bearer {self.api_key}",
            "Content-Type": "application/json",
        }
        data = {
            "model": self.model_name,
            "messages": [{"role": "user", "content": prompt}],
            "temperature": self.temperature,
            "max_tokens": self.max_tokens,
            "top_p": self.top_p,
        }
        response = requests.post(url, headers=headers, json=data)
        response.raise_for_status()
        return response.json()["choices"][0]["message"]["content"]

# Instantiate LLM with your OpenRouter API key
llm = OpenRouterLLM(api_key="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")

# Custom prompt template to combine context and question clearly
template = """You are a helpful assistant who answers questions based on the company policy excerpt provided.

Policy excerpt:
{context}

Question:
{question}

Answer:"""

prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=template,
)

# Create RetrievalQA pipeline with prompt
qa = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vectorstore.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt}
)

# Take user input question
question = input("Ask a question about the company policy: ")

# Run the QA system
output = qa.invoke({"query": question})

answer = output["result"]
sources = output.get("source_documents", [])

print("\nQuestion:", question)
print("Answer:", answer)
print("\nSource document(s):")
for doc in sources:
    print("-", doc.page_content)
